In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
import seaborn as sns
from scipy import stats
import utils
import constants
import scipy.stats as stats

In [2]:
# Define the list of swarm IDs and trial IDs
swarm_ids = [1, 2, 3, 4, 5, 6, 7, 8]
trial_ids = [1, 2, 3]

# Initialize an empty list to store dataframes
dfs = []

# Loop over swarm IDs and trial IDs
for swarm_id in swarm_ids:
    for trial_id in trial_ids:
        trajectory_file = os.path.join("swarm_data", f"Swarm_{swarm_id}", f"Swarm{swarm_id}_Trial{trial_id}", f"Swarm{swarm_id}_Trial{trial_id}_correct_order.csv")

        if os.path.exists(trajectory_file):
            df = pd.read_csv(trajectory_file)
            df['swarm_id'] = swarm_id
            df['trial_id'] = trial_id
            df.rename(columns={'track': 'bot_id'}, inplace=True)
            df['speed'] = np.sqrt((df.groupby('bot_id')['x'].diff() ** 2) + (df.groupby('bot_id')['y'].diff() ** 2))
            
            dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df.drop(columns=['ignore', 'track_fixed'], inplace=True)

frames = np.arange(1, 129)  # Assuming frames are from 1 to 129
bot_ids = range(1, 5)       # Assuming bot_ids are from 1 to 4

# Create a MultiIndex from product of frames and bot_ids
index = pd.MultiIndex.from_product([frames, bot_ids], names=['frame', 'bot_id'])

# Reindex the dataframe to include all possible frames and bot_ids
complete_dfs = []

for swarm_id in swarm_ids:
    for trial_id in trial_ids:

        sub_df = df[(df['swarm_id'] == swarm_id) & (df['trial_id'] == trial_id)]
        sub_df = sub_df.set_index(['frame', 'bot_id']).reindex(index, fill_value=np.nan).reset_index()
        sub_df['swarm_id'] = swarm_id
        sub_df['trial_id'] = trial_id
        complete_dfs.append(sub_df)

df = pd.concat(complete_dfs, ignore_index=True)

In [3]:
def evaluate_fitness(all_coords):
    # Create a boundary 

    min_y = 0
    max_y = 736

    x_starts = []
    x_starts = [all_coords[0,0], all_coords[128,0], all_coords[256,0], all_coords[384,0]]
    center_x = np.mean(x_starts)

    min_x = center_x - 736/2 #center eval area on x
    max_x = center_x + 736/2

    unique_points = np.unique(all_coords,axis=0)

    # true if in bounds
    x_min_mask = np.reshape(unique_points[:,0]>min_x,newshape=(-1,1))
    x_max_mask = np.reshape(unique_points[:,0]<max_x,newshape=(-1,1))
    y_min_mask = np.reshape(unique_points[:,1]>min_y,newshape=(-1,1))
    y_max_mask = np.reshape(unique_points[:,1]<max_y,newshape=(-1,1))

    mask = np.all(np.concatenate((x_min_mask, x_max_mask, y_min_mask, y_max_mask),axis=1),axis=1)

    unique_points_in_bounds = unique_points[mask]

    return fractal_box_count(unique_points_in_bounds,(min_x,min_y,max_x,max_y))


def fractal_box_count(points, boundary):
    # https://francescoturci.net/2016/03/31/box-counting-in-numpy/

    min_x,min_y,max_x,max_y = boundary # unpack tuple

    Ns=[]

    # scales = np.arange(start=2, stop=int(constants.BOUNDARY_LENGTH/constants.MIN_GRID_DIM)) #start with quadrents and go to resolution of voxcraft float
    levels = np.arange(start=1, stop=13)

    for level in levels: 

        scale = 2**level

        cell_width = 736/scale
        cell_height = 736/scale

        # H, edges=np.histogramdd(points, bins=(np.linspace(min_x,max_x,constants.BOUNDARY_LENGTH/scale),np.linspace(min_y,max_y,constants.BOUNDARY_LENGTH/scale)))
        H, edges=np.histogramdd(points, bins=(np.linspace(min_x,max_x,num=scale+1),np.linspace(min_y,max_y,num=scale+1)))

        weight = (cell_width*cell_height)/(736*736) # David scaling
        # print(level, weight)
        Ns.append(np.sum(H>0)*weight)

    # Divide by # of levels to get a value between 0-1
    scaled_box_count = np.sum(Ns)/len(levels) # David scaling
    # print(scaled_box_count)
    return scaled_box_count


In [4]:
fitness_records = []
for swarm in swarm_ids: 
    for trial in trial_ids: 
        fitness  = evaluate_fitness(utils.concatenate_bot_points(df, swarm, trial))
        fitness_records.append({'swarm_id': swarm, 'trial_id': trial, 'fitness': fitness})
fitness_df = pd.DataFrame(fitness_records)

# Merge the fitness DataFrame with the original DataFrame
df = pd.merge(df, fitness_df, on=['swarm_id', 'trial_id'], how='left')

In [5]:
mean_fitness_per_swarm = fitness_df.groupby('swarm_id')['fitness'].mean()
total_frames_speed_gt_0_5_or_nan = df[(df['speed'] > 1.0) | (df['speed'].isna())].groupby(['swarm_id', 'trial_id']).size().reset_index(name='total_frames')['total_frames']

fitness_df['motile_frames'] = np.array(total_frames_speed_gt_0_5_or_nan)

average_fitness_df = pd.DataFrame({'swarm_id': mean_fitness_per_swarm.index, 'fitness': mean_fitness_per_swarm.values, 'trial_id': 'A'})

fitness_df = pd.concat([fitness_df, average_fitness_df], ignore_index=True)
fitness_df['swarm_trial'] = fitness_df['swarm_id'].astype(str) + '-' + fitness_df['trial_id'].astype(str)

fitness_df = fitness_df.sort_values(by='swarm_trial')

In [6]:
fitness_df['fitness_per_frame'] = fitness_df['fitness'] / fitness_df['motile_frames']

# Calculate average 'fitness_per_frame' for each swarm
average_fitness_per_swarm = fitness_df.groupby('swarm_id')['fitness_per_frame'].median()

# Assign the average values to corresponding 'trial_id' 'A' rows within each swarm
for swarm_id, avg_fitness in average_fitness_per_swarm.items():
    mask = (fitness_df['swarm_id'] == swarm_id) & (fitness_df['trial_id'] == 'A')
    fitness_df.loc[mask, 'fitness_per_frame'] = avg_fitness

In [7]:
#p value with medians 
evolved_swarms = [1, 2, 6, 8]
random_swarms = [3, 4, 5, 7]
# Filter and group by 'swarm_id' for group1
evolved_medians = fitness_df[(fitness_df['swarm_id'].isin(evolved_swarms)) & 
                            (fitness_df['trial_id'] !='A')].groupby('swarm_id')['fitness_per_frame'].median()

# Filter and group by 'swarm_id' for group2
random_medians = fitness_df[(fitness_df['swarm_id'].isin(random_swarms)) & 
                            (fitness_df['trial_id'] !='A')].groupby('swarm_id')['fitness_per_frame'].median()

alternative = 'greater'
u_statistic, p_value = stats.mannwhitneyu(evolved_medians, random_medians, alternative=alternative)
print(f'p-value w/alternative {alternative}: {p_value}')

p-value w/alternative greater: 0.02857142857142857
